In [ ]:
!pip install wget
import wget

wget.download("https://github.com/TingYu-Shiu/deeplearning/blob/main/one_piece_mini.zip")

'one_piece_mini (33).zip'

In [ ]:
import zipfile

with zipfile.ZipFile("one_piece_mini.zip","r") as zip_file:
  zip_file.extractall("one_piece_mini")

In [ ]:
from PIL import Image
import numpy as np
from pathlib import Path

train_path = Path("one_piece_mini") / "train"
test_path = Path("one_piece_mini") / "test"

x = [i for i in train_path.rglob("*") if i.is_file()]
z = [j for j in test_path.rglob("*") if j.is_file()]

train_list = []
train_label = []

test_list = []
test_label = []

target_width = 64  # 目标宽度
target_height = 64  # 目标高度

with open(Path("one_piece_mini") / "classnames.txt", "r") as f:
    lines = f.readlines()
labels = [line.strip() for line in lines]

for i in x:
    image1 = Image.open(i)
    image1 = image1.convert("RGB")

    # 数据增强 - 随机水平翻转
    if np.random.rand() < 0.5:
        image1 = image1.transpose(Image.FLIP_LEFT_RIGHT)

    # 数据增强 - 随机旋转
    angle = np.random.randint(-10, 10)
    image1 = image1.rotate(angle)

    resized_image = image1.resize((target_width, target_height))
    image_array = np.array(resized_image) / 255
    train_list.append(image_array)
    train_label.append(labels.index(i.parent.name))

train_array = np.array(train_list)

for k in z:
    image2 = Image.open(k)
    image2 = image2.convert("RGB")

    # 数据增强 - 随机水平翻转
    if np.random.rand() < 0.5:
        image2 = image2.transpose(Image.FLIP_LEFT_RIGHT)

    # 数据增强 - 随机旋转
    angle = np.random.randint(-10, 10)
    image2 = image2.rotate(angle)

    resized_image1 = image2.resize((target_width, target_height))
    image_array1 = np.array(resized_image1) / 255
    test_list.append(image_array1)
    test_label.append(labels.index(k.parent.name))

test_array = np.array(test_list)

In [ ]:
import numpy as np
from torch.utils.data import DataLoader, Dataset


# 自訂資料集類別
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # 返回資料樣本及其對應的標籤
        return self.data[index], self.labels[index]

# 創建資料集實例
train_dataset = CustomDataset(train_array, train_label)
test_dataset = CustomDataset(test_array, test_label)
# 創建資料載入器
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
len(train_dataset),len(test_dataset)

(240, 60)

In [ ]:
from torch import nn
import torch
import matplotlib.pyplot as plt

In [ ]:
from torch import nn
class ImageClassificationModel3(nn.Module):
  def __init__(self, input_shape, output_shape):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
      nn.Conv2d(in_channels=input_shape,
          out_channels=8,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.Conv2d(in_channels=8,
          out_channels=8,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2, 2),
            stride=2,
            padding=0
      )
    )

    self.conv_block_2 = nn.Sequential(
      nn.Conv2d(in_channels=8,
          out_channels=16,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.Conv2d(in_channels=16,
          out_channels=16,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2, 2),
            stride=2,
            padding=0
      )
    )

    self.classifier = nn.Sequential(
      nn.Flatten(start_dim=1, end_dim=-1),
      nn.Linear(in_features=16*16*16, out_features=output_shape)
    )
  def forward(self, x):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

In [ ]:
model = ImageClassificationModel3(3, 3)

In [ ]:
cost_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
def accuracy_fn(y_pred, y_true):
  correct_num = (y_pred == y_true).sum()
  acc = correct_num / len(y_true) *100
  return acc

In [ ]:
def train_step(dataloader,model,cost_fn,optimizer,accuracy_fn):
  train_cost = 0
  train_acc = 0
  for batch, (x,y) in enumerate(dataloader):
    #x = x.to(device)
    #y = y.to(device)
    x = x.permute(0, 3, 1, 2)
    x = x.to(torch.float)
    y = y.to(torch.long)

    lambda_reg = 1

    model.train()

    y_pred = model(x)
    cost = cost_fn(y_pred, y)
    train_cost += cost
    train_acc += accuracy_fn(y_pred.argmax(dim=1),y)

    optimizer.zero_grad()


    reg_loss = 0
    for param in model.parameters():
        reg_loss += torch.norm(param, p=2)/len(x)  # 使用 L2 正則化
    cost += lambda_reg * reg_loss


    cost.backward()
    optimizer.step()

  train_cost /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  print(f"Train Cost: {train_cost:.4f},Train Acc: {train_acc:.2f} ")
  return train_cost

In [ ]:
def test_step(dataloader, model, cost_fn, accuracy_fn):
  test_cost = 0
  test_acc = 0
  model.eval()
  with torch.inference_mode():
    for x, y in dataloader:
      #x = x.to(device)
      #y = y.to(device)
      x = x.permute(0, 3, 1, 2)
      x = x.to(torch.float)
      y = y.to(torch.long)

      test_pred = model(x)

      test_cost += cost_fn(test_pred, y)
      test_acc += accuracy_fn(test_pred.argmax(dim=1), y)

    test_cost /= len(test_dataloader)
    test_acc /= len(test_dataloader)

  print(f"Test Cost: {test_cost:.4f}, Test Acc: {test_acc:.2f} \n")

In [ ]:
from tqdm.auto import tqdm

epochs = 50

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-------")

  train_step(train_dataloader, model, cost_fn, optimizer, accuracy_fn)

  test_step(test_dataloader, model, cost_fn, accuracy_fn)

  from tqdm.auto import tqdm



  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0
-------
Train Cost: 1.1078,Train Acc: 32.03 
Test Cost: 1.0993, Test Acc: 33.04 

Epoch: 1
-------
Train Cost: 1.1005,Train Acc: 28.12 
Test Cost: 1.0922, Test Acc: 35.71 

Epoch: 2
-------
Train Cost: 1.0933,Train Acc: 45.31 
Test Cost: 1.0870, Test Acc: 55.36 

Epoch: 3
-------
Train Cost: 1.0812,Train Acc: 50.39 
Test Cost: 1.0662, Test Acc: 53.12 

Epoch: 4
-------
Train Cost: 1.0575,Train Acc: 45.31 
Test Cost: 1.0279, Test Acc: 43.53 

Epoch: 5
-------
Train Cost: 0.9894,Train Acc: 62.50 
Test Cost: 0.9815, Test Acc: 54.24 

Epoch: 6
-------
Train Cost: 0.8977,Train Acc: 64.45 
Test Cost: 0.9824, Test Acc: 52.46 

Epoch: 7
-------
Train Cost: 0.8125,Train Acc: 65.62 
Test Cost: 0.9474, Test Acc: 61.83 

Epoch: 8
-------
Train Cost: 0.7738,Train Acc: 66.02 
Test Cost: 0.9608, Test Acc: 63.17 

Epoch: 9
-------
Train Cost: 0.7304,Train Acc: 72.27 
Test Cost: 0.9619, Test Acc: 62.95 

Epoch: 10
-------
Train Cost: 0.6890,Train Acc: 71.88 
Test Cost: 1.0299, Test Acc: 58.26 